<a href="https://colab.research.google.com/github/soheilpaper/-tft-2.4-ili9341-STM32/blob/master/notebook/scrape_and_analyze_with_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/easonlai/google_news_content_scrape_and_analyze_with_gpt/

fatal: destination path 'google_news_content_scrape_and_analyze_with_gpt' already exists and is not an empty directory.


In [3]:
%cd ./google_news_content_scrape_and_analyze_with_gpt
!ls

/content/google_news_content_scrape_and_analyze_with_gpt
data  git-images  README.md  scrape_and_analyze_with_gpt.ipynb


In [4]:
!pip install GoogleNews

In [5]:
!pip install fake_useragent
!pip install newspaper3k

In [6]:
!     pip install lxml[html_clean]

In [7]:
# Import necessary libraries
from GoogleNews import GoogleNews
import pandas as pd
import requests
from fake_useragent import UserAgent
import newspaper
from newspaper import fulltext
import re

In [8]:
# Define the keyword to search.
keyword = 'grilfriend long distance'

In [9]:
# Perform news scraping from Google and extract the result into Pandas dataframe.
googlenews = GoogleNews(lang='en', region='US', period='1d', encode='utf-8')
googlenews.clear()
googlenews.search(keyword)
googlenews.get_page(2)
news_result = googlenews.result(sort=True)
news_data_df = pd.DataFrame.from_dict(news_result)

'<' not supported between instances of 'float' and 'datetime.datetime'


In [10]:
# Display information of dataframe.
news_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   title     22 non-null     object        
 1   media     22 non-null     object        
 2   date      22 non-null     object        
 3   datetime  20 non-null     datetime64[ns]
 4   desc      22 non-null     object        
 5   link      22 non-null     object        
 6   img       22 non-null     object        
dtypes: datetime64[ns](1), object(6)
memory usage: 1.3+ KB


In [11]:
# Display header of dataframe.
news_data_df.head()

,title,media,date,datetime,desc,link,img
0,,,,NaT,,/search?lr=lang_en&sca_esv=56593433f3efb7e1&hl...,
1,,,,NaT,,/search?lr=lang_en&sca_esv=56593433f3efb7e1&hl...,
2,42 long-distance relationship gifts to keep ea...,Business Insider,2 days ago,2025-05-29 08:07:59.324582,Our picks for the best gifts for long-distance...,https://www.businessinsider.com/guides/gifts/b...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
3,Country Music Artist Marries Long-Distance Gir...,PopCulture.com,1 week ago,2025-05-24 08:07:59.340002,Jamey Johnson is officially off the market aft...,https://popculture.com/country-music/news/coun...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
4,“My biggest motivational factor” - Johan Ghaza...,Sportskeeda,2 weeks ago,2025-05-17 08:07:59.370542,Johan Ghazali opened up about the support he's...,https://www.sportskeeda.com/mma/news-my-bigges...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."


In [12]:
news_data_df_with_text = []
for index, headers in news_data_df.iterrows():
    news_title = str(headers['title'])
    news_media = str(headers['media'])
    news_update = str(headers['date'])
    news_timestamp = str(headers['datetime'])
    news_description = str(headers['desc'])
    news_link = str(headers['link'])
    print(news_link)
    news_img = str(headers['img'])
    try:
        html = requests.get(news_link, headers={'User-Agent':ua.chrome}, timeout=5).text
        text = fulltext(html)
        print('Text Content Scraped')
    except:
        print('Text Content Scraped Error, Skipped')
        text = ""   # Always define text, even if scraping fails
    news_data_df_with_text.append([news_title, news_media, news_update, news_timestamp,
                                   news_description, news_link, news_img, text])

news_data_with_text_df = pd.DataFrame(news_data_df_with_text, columns=['Title', 'Media', 'Update', 'Timestamp',
                                                                       'Description', 'Link', 'Image', 'Text'])

/search?lr=lang_en&sca_esv=56593433f3efb7e1&hl=en&gl=en&biw=1920&bih=976&tbs=lr:lang_1en,qdr:1d,sbd:1&tbm=nws&q=girlfriend+long+distance&spell=1
Text Content Scraped Error, Skipped
/search?lr=lang_en&sca_esv=56593433f3efb7e1&hl=en&gl=en&biw=1920&bih=976&tbs=lr:lang_1en,qdr:1d,sbd:1&tbm=nws&q=grilfriend+long+distance&nfpr=1&sa=X&ved=2ahUKEwj28Ji6oM2NAxUPfTABHYPDIQ0QvgV6BAgLEAI
Text Content Scraped Error, Skipped
https://www.businessinsider.com/guides/gifts/best-long-distance-relationship-gifts&ved=2ahUKEwj28Ji6oM2NAxUPfTABHYPDIQ0QxfQBegQIBBAC&usg=AOvVaw1D3JoTxY-ogfdLpHfh5IbO
Text Content Scraped Error, Skipped
https://popculture.com/country-music/news/country-music-artist-marries-long-distance-girlfriend-in-wedding-attended-by-big-country-stars/&ved=2ahUKEwj28Ji6oM2NAxUPfTABHYPDIQ0QxfQBegQIChAC&usg=AOvVaw0-DsF6Y8rGYwbGK6-rTrw6
Text Content Scraped Error, Skipped
https://www.sportskeeda.com/mma/news-my-biggest-motivational-factor-johan-ghazali-says-long-distance-relationship-girlfriend-s

In [13]:
# Display the entire dataframe for sample checking.
news_data_with_text_df

,Title,Media,Update,Timestamp,Description,Link,Image,Text
0,,,,NaT,,/search?lr=lang_en&sca_esv=56593433f3efb7e1&hl...,,
1,,,,NaT,,/search?lr=lang_en&sca_esv=56593433f3efb7e1&hl...,,
2,42 long-distance relationship gifts to keep ea...,Business Insider,2 days ago,2025-05-29 08:07:59.324582,Our picks for the best gifts for long-distance...,https://www.businessinsider.com/guides/gifts/b...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",
3,Country Music Artist Marries Long-Distance Gir...,PopCulture.com,1 week ago,2025-05-24 08:07:59.340002,Jamey Johnson is officially off the market aft...,https://popculture.com/country-music/news/coun...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",
4,“My biggest motivational factor” - Johan Ghaza...,Sportskeeda,2 weeks ago,2025-05-17 08:07:59.370542,Johan Ghazali opened up about the support he's...,https://www.sportskeeda.com/mma/news-my-bigges...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",
5,Eamonn 'lonely' as new girlfriend tells him: I...,Daily Mail,2 weeks ago,2025-05-17 08:07:59.354160,Eamonn Holmes surprised fans when it emerged h...,https://www.dailymail.co.uk/tvshowbiz/article-...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",
6,Brad Pitt 'Doing All He Can' to Save Long-Dist...,RadarOnline,1 month ago,2025-04-30 08:07:59.574014,Brad Pitt's relationship may not survive as he...,https://radaronline.com/p/brad-pitt-save-relat...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",
7,"Column | Asking Eric: Son’s older, internet re...",The Washington Post,1 month ago,2025-04-30 08:07:59.551862,Their 22-year-old son has fallen for a 36-year...,https://www.washingtonpost.com/advice/2025/04/...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",
8,How To Apologize To Your Girlfriend: 25+ Simpl...,MomJunction,1 month ago,2025-04-30 08:07:59.528822,Have you hurt her feelings with your words and...,https://www.momjunction.com/articles/how-to-ap...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",
9,500+ Engaging Long-Distance Relationship Quest...,MomJunction,1 month ago,2025-04-30 08:07:59.472155,A long-distance relationship makes you miss yo...,https://www.momjunction.com/articles/long-dist...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",


In [14]:
# Save the result dataframe into a CSV file.
news_data_with_text_df.to_csv("./data/news_data_with_text.csv")

In [15]:
# Reload the saved news data content from a CSV file.
news_data_with_text_df = pd.read_csv("./data/news_data_with_text.csv",  index_col=0)

In [16]:
# Select the top 10 news records (the latest 10) for summarization.
news_data_with_text_df = news_data_with_text_df.head(10)

In [17]:
# Display the scraped text content from the first news record.
print(news_data_with_text_df['Text'].values[0])

nan


In [24]:
import re

# Get the first record
text_data = news_data_with_text_df['Text'].iloc[0]

# Decode if it's bytes
if isinstance(text_data, bytes):
    text_data = text_data.decode('utf-8')  # or use 'latin1' if utf-8 fails

# Now count words
word_count = len(re.findall(r'\w+', text_data))
print(f"Word count: {word_count}")

TypeError: cannot use a string pattern on a bytes-like object

In [21]:



%%writefile participants.csv
First Name,Last Name,Email,Extra Info
Shoaib,Khanmohammadi,sh.khanmohammadi@kut.ac.ir,
Asghari,,Asghari_1@yahoo.com,
Kobra,Shojaeizadeh,shojaeizadeh.1996@yahoo.com,
Mohammadreza,Nadami Ardastani,mhmdrdanadmyardstany@gmail.com,
Faezeh,Ataei,,
Saleh,Hasannezhad,salehhasannezhad@gmail.com,
Amirhossein,Zekri,,
Zahra,Zharfimehr,,
Yousef,Abedi,Yusuf.Edaram@gmail.com,
Somayeh,Moradhasseli,,
Raheleh,Jalalniya,,
Mahtab,Seyedmirzaei,,Senior Electrical Engineering undergraduate
M Danesh,,
Efat,Heidari,,
Navid,Dehghani,,
Mehda,Mortazavi,,
Kamali,,m kamal heidari@gmail.com,
Simin,Haghnazari,haghnazari.simin@gmail.com,
Mohsen,Parvish,mohsen.parvish@yahoo.com,
Faezeh,Ataei,Faezeh.ata82@gmail.com,
Mahsa,Taslimi,taslimi_mahsa@yahoo.com,
Mohammad Mahdi,Bashiri,hjnnj74@gmail.com,
Dr,Rezapour,m.rezapour@modares.ac.ir,
Hajipoor,,haji200655@gmail.com,
Raheleh,Jalalniya,r.1996.jalal@gmail.com,
Farnaz,Ershadfath,,
Kiana,Momeni,Kianamomeni.8383@gmail.com,
Jafar,Ebrahimifar,,
Hamed,Bakhshandeh,hbakhshandeh71@gmail.com,
Dordaneh,Aghajani,Aghajani.d@gmail.com,
Saleh,Hasannezhad,salehhasannezhad@gmail.com,
Shoaib,Khanmohammadi,sh.khanmohammadi@kut.ac.ir,
Asghari,,Asghari_1@yahoo.com,
Kobra,Shojaeizadeh,shojaeizadeh.1996@yahoo.com,
Mahtab,Seyedmirzaei,,LinkedIn: https://www.linkedin.com/in/mahtab-seyedmirzaei/
Behroz,Dolatparast,,From Hamedan
Hassan,Talayi,,Sari
Hossein,Arabi Jashoghani,hossein.arabi.j1258@gmail.com,
Zeinab,Asadi,,
Faezeh,Dehghani,Faeze.dehqaan@gmail.com,
Yeganeh,Karami,yeganehkarami2001@gmail.com,
Sahar,Mozaffarian,Sahar_m79@yahoo.com,
Fatemeh,Shateri,baharsha76@gmail.com,
Soheil,,soheil.power@gmail.com

TypeError: cannot use a string pattern on a bytes-like object

In [22]:


# Concatenate all the top 10 news record's text content into a single string.
news_text_content_string = news_data_with_text_df.to_string(columns=['Text'], header=False, index=False)
print(news_text_content_string)

NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN
NaN


In [ ]:
# Count the number of words in the top 10 news record's text content.
len(re. findall(r'\w+', news_text_content_string))

In [ ]:
# Import necessary libraries
import openai
import platform
import os

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [ ]:
# Define the function to count the number of tokens.
def count_tokens(stringname):
    tokens = word_tokenize(stringname)
    return len(tokens)

In [ ]:
# Display the number of tokens from the top 10 news record's text content.
stringname = news_text_content_string

token_count = count_tokens(stringname)
print(f"Number of tokens: {token_count}")

In [ ]:
# Define the function to divide the top 10 news record's text content into segments of 2000 tokens
# with the overlap of 100 tokens to avoid losing information from the split.
def break_up_file(tokens, chunk_size, overlap_size):
    if len(tokens) <= chunk_size:
        yield tokens
    else:
        chunk = tokens[:chunk_size]
        yield chunk
        yield from break_up_file(tokens[chunk_size-overlap_size:], chunk_size, overlap_size)

def break_up_file_to_chunks(stringname, chunk_size=2000, overlap_size=100):
    tokens = word_tokenize(stringname)
    return list(break_up_file(tokens, chunk_size, overlap_size))

In [ ]:
# Trial run of dividing function.
stringname = news_text_content_string

chunks = break_up_file_to_chunks(stringname)
for i, chunk in enumerate(chunks):
    print(f"Chunk {i}: {len(chunk)} tokens")

In [ ]:
# Define the function to convert tokenized text back to normal text prompts.
def convert_to_detokenized_text(tokenized_text):
    prompt_text = " ".join(tokenized_text)
    prompt_text = prompt_text.replace(" 's", "'s")
    return prompt_text

In [ ]:
# Configure the baseline configuration of the OpenAI library.
openai.api_type = "azure"
openai.api_base = "https://PLESAE_ENTER_YOUR_OWNED_AOAI_RESOURCE_NAME.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = "PLEASE_ENTER_YOUR_OWNED_AOAI_SERVICE_KEY"

In [ ]:
# Perform news text content summarization by Azure OpenAI Service (GPT3) for each chunk.
stringname = news_text_content_string

prompt_response = []
chunks = break_up_file_to_chunks(stringname)

for i, chunk in enumerate(chunks):
    print("Processing chunk " + str(i))
    prompt_request = "Summarize this news content: " + convert_to_detokenized_text(chunks[i])
    response = openai.Completion.create(
            engine="eason-text-davinci-002",
            prompt=prompt_request,
            temperature=.5, # Default is 1.
            max_tokens=500,
            top_p=1 # Default is 0.5.
    )

    prompt_response.append(response["choices"][0]["text"].strip())

In [ ]:
# Define the prompt to perform summarization into 1,500 words for each summarized content.
prompt_request = "Consolidate these news content summaries into 1500 words sentences: " + str(prompt_response)

In [ ]:
# Perform summarization by Azure OpenAI Service (GPT3) for each chunk of summarized content.
response = openai.Completion.create(
        engine="PLEASE_ENTER_YOUR_AOAI_MODEL_DEPLOYMENT_NAME",
        prompt=prompt_request,
        temperature=.5, # Default is 1.
        max_tokens=1000,
        top_p=1 # Default is 0.5.
    )

In [ ]:
# Display the final summary from the top 10 news record's text content.
news_content_summary = response["choices"][0]["text"].strip()
print(news_content_summary)

In [ ]:
# Select the necessary columns and convert the dataframe into HTML for showing in the email.
news_data_with_text_df_1 = news_data_with_text_df[["Title", "Media", "Timestamp", "Description", "Link"]]
news_data_with_text_df_1_html = news_data_with_text_df_1.to_html(index=False)

In [ ]:
# Email the GPT news summary with the news source reference table via MailJet.
from mailjet_rest import Client
import os
api_key = 'PLEASE_ENTER_YOUR_OWNED_MAILJET_API_KEY_NAME'
api_secret = 'PLEASE_ENTER_YOUR_OWNED_MAILJET_API_KEY_SECRET'
mailjet = Client(auth=(api_key, api_secret), version='v3.1')
data = {
  'Messages': [
    {
      "From": {
        "Email": "easonlai888@gmail.com",
        "Name": "Eason"
      },
      "To": [
        {
          "Email": "easonlai888@gmail.com",
          "Name": "Eason"
        }
      ],
      "Subject": "GPT News Summary of Today",
      "HTMLPart": "<h3>Here is the news summary of GPT for today.</h3>{}<br><br> \
                   <h3>GPT News Summary Sources</h3>{}" \
                   .format(news_content_summary, news_data_with_text_df_1_html),
    }
  ]
}
result = mailjet.send.create(data=data)

print(result.status_code)
print(result.json())
